## Bokeh Demo

https://bokeh.pydata.org/en/latest/

Interactive visualization with Python, designed for web browsers! Explore the gallery for more examples: https://bokeh.pydata.org/en/latest/docs/gallery.html#gallery

The basic steps to creating plots with the bokeh.plotting interface are:

 * Prepare some data
  * Can be plain python lists, NumPy arrays, or Pandas series.
 * Tell Bokeh where to generate output
  * can use output_file() or output_notebook() for use in Jupyter notebooks.
 * Call figure()
  * This creates a plot with typical default options and easy customization of title, tools, and axes labels.
 * Add renderers
  * In this case, we use line() for our data, specifying visual customizations like colors, legends and widths.
 * Ask Bokeh to show() or save() the results.
  * These functions save the plot to an HTML file and optionally display it in a browser.


Bokeh can also be used to create map-based visualizations. In this tutorial we will use Google Maps API.

First things first: we're going to need a Google Maps Developer key. Sign up for one here: https://developers.google.com/maps/documentation/javascript/get-api-key

Save your API key somewhere secure - you're going to need it soon.

Now, we import our dataset into a pandas dataframe.

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('./data/metadata.csv') 

In [4]:
data.head()

,Cell Cgi,Cell Tower Location,Comm Identifier,Comm Timedate String,Comm Type,Latitude,Longitude
0,50501015388B9,REDFERN TE,f1a6836c0b7a3415a19a90fdd6f0ae18484d6d1e,4/1/14 9:40,Phone,-33.892933,151.202296
1,50501015388B9,REDFERN TE,62157ccf2910019ffd915b11fa037243b75c1624,4/1/14 9:42,Phone,-33.892933,151.202296
2,505010153111F,HAYMARKET #,c8f92bd0f4e6fb45ed7fce96fc831b283db2b642,4/1/14 13:13,Phone,-33.880329,151.205690
3,505010153111F,HAYMARKET #,f1a6836c0b7a3415a19a90fdd6f0ae18484d6d1e,4/1/14 13:13,Phone,-33.880329,151.205690
4,5.05E+106,HAYMARKET #,f1a6836c0b7a3415a19a90fdd6f0ae18484d6d1e,4/1/14 17:27,Phone,-33.880329,151.205690


Install bokeh and import packages: 

In [5]:
!pip install bokeh

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
from bokeh.io import output_file, show, save, curdoc
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, Range1d, PanTool, 
    WheelZoomTool, BoxSelectTool, LinearColorMapper, ContinuousColorMapper, HoverTool,
    Plot, Circle, LinearAxis, Text,
    SingleIntervalTicker, Slider, CustomJS, Select
)
from bokeh.palettes import Blues9 as blu
from bokeh.layouts import column, row, widgetbox
from bokeh.models.widgets import Slider

In [8]:
data.columns

Index(['Cell Cgi', 'Cell Tower Location', 'Comm Identifier',
       'Comm Timedate String', 'Comm Type', 'Latitude', 'Longitude'],
      dtype='object')

In [12]:
# set the data source - we'll pick certain columns from the dataframe that we want to visualize on a map

source = ColumnDataSource(data={
    'long'  : data['Longitude'],
    'lat'   : data['Latitude'],
    'loc': data['Cell Tower Location'],
    'timedate': data['Comm Timedate String'],
    'type': data['Comm Type']
})

In [13]:
# let's see what the lats and longs look like, to determine where to locate our map

data.describe()

,Latitude,Longitude
count,10476.000000,10476.000000
mean,-35.136188,150.612841
std,3.141723,1.470169
min,-42.884810,144.848243
25%,-33.884603,151.202296
50%,-33.796610,151.266540
75%,-33.788150,151.266540
max,-33.557310,151.289340


In [14]:
# set map options - lat and long is in the center so we'll start with about halfway between the min and max

map_options = GMapOptions(lat=-38, lng=148, map_type="roadmap", zoom=16)

In [9]:
# initiate our plot with the map options we just defined

plot = GMapPlot(x_range=Range1d(), y_range=Range1d(), 
                map_options=map_options, plot_width=900, plot_height=800)

plot.title.text = "cell tower data"

# FILL IN YOUR API KEY HERE! OR BETTER YET - SAVE IT IN AN EXTERNAL FILE (IN THE SAME FOLDER) AND REFERENCE HERE
plot.api_key = './my_google_key' 

# map colors to comm type
mapper = ContinuousColorMapper(
    palette=blu,
    # set colors/use with data['type']
)

# define circles
circle = Circle(x="long", y="lat", size='' # some count, 
                fill_color={'field': '', 'transform': mapper}, 
                fill_alpha=1, line_color=None)

# add circles and source to plot
plot.add_glyph(source, circle)

# add interactive tools to plot
hover = HoverTool(tooltips=[("datetime", '@datetime'), 
                    ("location", '@loc')],
                    )

plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool(), hover)


# Make a slider object
slider = Slider(start=0, end=23, value=0, step=1, title="Time")

# make a dropdown object
select = Select(
    options=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], 
    value='Monday', title="Day")


# Define the callback function
def update_plot(attr, old, new):
    new_h = slider.value
    new_hour = {
             'long'  : hours_aps_ord.loc[hours_aps_ord['hour']==new_h].long,
             'lat'   : hours_aps_ord.loc[hours_aps_ord['hour']==new_h].lat,
             'clients' : hours_aps_ord.loc[hours_aps_ord['hour']==new_h].client_mac_address,
             'size': hours_aps_ord.loc[hours_aps_ord['hour']==new_h].client_mac_address_sized,
             'time': hours_aps_ord.loc[hours_aps_ord['hour']==new_h].session_time_m,
             'ap': hours_aps_ord.loc[hours_aps_ord['hour']==new_h].ap_mac,
             'avg_time': hours_aps_ord.loc[hours_aps_ord['hour']==new_h].avg_time_client
         }

   
    source.data = new_hour
    
# Attach the callback to the 'value' property of slider
slider.on_change('value', update_plot)

def callack(attr, old, new):
    if new == 'Monday': 
        source.data = {
             'long'  : hours_aps_ord.loc[hours_aps_ord['date']==22].long,
             'lat'   : hours_aps_ord.loc[hours_aps_ord['date']==22].lat,
             'clients' : hours_aps_ord.loc[hours_aps_ord['date']==22].client_mac_address,
             'size': hours_aps_ord.loc[hours_aps_ord['date']==22].client_mac_address_sized,
             'time': hours_aps_ord.loc[hours_aps_ord['date']==22].session_time_m,
             'ap': hours_aps_ord.loc[hours_aps_ord['date']==22].ap_mac,
             'avg_time': hours_aps_ord.loc[hours_aps_ord['date']==22].avg_time_client
        }
    elif new == 'Tuesday': 
        source.data = {
             'long'  : hours_aps_ord.loc[hours_aps_ord['date']==23].long,
             'lat'   : hours_aps_ord.loc[hours_aps_ord['date']==23].lat,
             'clients' : hours_aps_ord.loc[hours_aps_ord['date']==23].client_mac_address,
             'size': hours_aps_ord.loc[hours_aps_ord['date']==23].client_mac_address_sized,
             'time': hours_aps_ord.loc[hours_aps_ord['date']==23].session_time_m,
             'ap': hours_aps_ord.loc[hours_aps_ord['date']==23].ap_mac,
             'avg_time': hours_aps_ord.loc[hours_aps_ord['date']==23].avg_time_client
        }
    elif new == 'Wednesday': 
        source.data = {
             'long'  : hours_aps_ord.loc[hours_aps_ord['date']==24].long,
             'lat'   : hours_aps_ord.loc[hours_aps_ord['date']==24].lat,
             'clients' : hours_aps_ord.loc[hours_aps_ord['date']==24].client_mac_address,
             'size': hours_aps_ord.loc[hours_aps_ord['date']==24].client_mac_address_sized,
             'time': hours_aps_ord.loc[hours_aps_ord['date']==24].session_time_m,
             'ap': hours_aps_ord.loc[hours_aps_ord['date']==24].ap_mac,
             'avg_time': hours_aps_ord.loc[hours_aps_ord['date']==24].avg_time_client
        }
    elif new == 'Thursday': 
        source.data = {
             'long'  : hours_aps_ord.loc[hours_aps_ord['date']==25].long,
             'lat'   : hours_aps_ord.loc[hours_aps_ord['date']==25].lat,
             'clients' : hours_aps_ord.loc[hours_aps_ord['date']==25].client_mac_address,
             'size': hours_aps_ord.loc[hours_aps_ord['date']==25].client_mac_address_sized,
             'time': hours_aps_ord.loc[hours_aps_ord['date']==25].session_time_m,
             'ap': hours_aps_ord.loc[hours_aps_ord['date']==25].ap_mac,
             'avg_time': hours_aps_ord.loc[hours_aps_ord['date']==25].avg_time_client
        }
    elif new == 'Friday': 
        source.data = {
             'long'  : hours_aps_ord.loc[hours_aps_ord['date']==19].long,
             'lat'   : hours_aps_ord.loc[hours_aps_ord['date']==19].lat,
             'clients' : hours_aps_ord.loc[hours_aps_ord['date']==19].client_mac_address,
             'size': hours_aps_ord.loc[hours_aps_ord['date']==19].client_mac_address_sized,
             'time': hours_aps_ord.loc[hours_aps_ord['date']==19].session_time_m,
             'ap': hours_aps_ord.loc[hours_aps_ord['date']==19].ap_mac,
             'avg_time': hours_aps_ord.loc[hours_aps_ord['date']==19].avg_time_client
        }
    elif new == 'Saturday': 
        source.data = {
             'long'  : hours_aps_ord.loc[hours_aps_ord['date']==20].long,
             'lat'   : hours_aps_ord.loc[hours_aps_ord['date']==20].lat,
             'clients' : hours_aps_ord.loc[hours_aps_ord['date']==20].client_mac_address,
             'size': hours_aps_ord.loc[hours_aps_ord['date']==20].client_mac_address_sized,
             'time': hours_aps_ord.loc[hours_aps_ord['date']==20].session_time_m,
             'ap': hours_aps_ord.loc[hours_aps_ord['date']==20].ap_mac,
             'avg_time': hours_aps_ord.loc[hours_aps_ord['date']==20].avg_time_client
        }
    else:
        source.data = {
             'long'  : hours_aps_ord.loc[hours_aps_ord['date']==21].long,
             'lat'   : hours_aps_ord.loc[hours_aps_ord['date']==21].lat,
             'clients' : hours_aps_ord.loc[hours_aps_ord['date']==21].client_mac_address,
             'size': hours_aps_ord.loc[hours_aps_ord['date']==21].client_mac_address_sized,
             'time': hours_aps_ord.loc[hours_aps_ord['date']==21].session_time_m,
             'ap': hours_aps_ord.loc[hours_aps_ord['date']==21].ap_mac,
             'avg_time': hours_aps_ord.loc[hours_aps_ord['date']==21].avg_time_client
        }

select.on_change('value', callack)

# Make a layout of slider and plot and add it to the current document
layout = column(plot, select, slider)
curdoc().add_root(layout)

NameError: name 'hours_aps_ord' is not defined